In [1]:
import numpy as np
import skimage.io 
import matplotlib.pyplot as plt 
import os, sys
import pickle
import pandas as pd 
img_folder = '3seg+conjugated 8_26'

In [2]:
DEFAULT_CHANNELS = (1, 2, 3)

RGB_MAP = {
    1: {
        'rgb': np.array([255,0,0]),
        'range': [0, 40]
    },
    2: {
        'rgb': np.array([0,255,0]),
        'range': [0, 150]
    },
    3: {
        'rgb': np.array([0,0,255]),
        'range': [0, 80]
    }
}

def convert_to_rgb(t, channels=DEFAULT_CHANNELS, vmax=255, rgb_map=RGB_MAP):
    """
    Converts and returns the image data as RGB image
    Parameters
    ----------
    t : np.ndarray
        original image data
    channels : list of int
        channels to include
    vmax : int
        the max value used for scaling
    rgb_map : dict
        the color mapping for each channel
        See rxrx.io.RGB_MAP to see what the defaults are.
    Returns
    -------
    np.ndarray the image data of the site as RGB channels
    """
    dim1,dim2,_ = t.shape
    colored_channels = []
    for i, channel in enumerate(channels):
        x = (t[:, :, channel-1] / vmax) / \
            ((rgb_map[channel]['range'][1] - rgb_map[channel]['range'][0]) / 255) + \
            rgb_map[channel]['range'][0] / 255
        x = np.where(x > 1., 1., x)
        x_rgb = np.array(
            np.outer(x, rgb_map[channel]['rgb']).reshape(dim1,dim2, 3),
            dtype=int)
        colored_channels.append(x_rgb)
    im = np.array(np.array(colored_channels).sum(axis=0), dtype=int)
    im = np.where(im > 255, 255, im)
    im = im.astype(np.uint8)
    return im

In [3]:
# Read Images from Dapi and Phallaodin channel

cycle_name = 'Lg_2seg__40X8_26'

cycle_cond = 'Fw'
dapi_ch = 'CH1'
pha_ch = 'CH3'
wga_ch = 'CH4'

img_ch = {}

for (dirpath, dirnames, filenames) in os.walk(img_folder):
    for name in sorted(filenames):
        if cycle_name in dirpath and 'tif' in name:
            
            # Get information from image name 
            condition = dirpath.split(cycle_name)[-1]
            z_stack = '_'.join(name.split('_')[:3])
            channel = name.split('_')[-1].split('.')[0]
            
            # Add key if doesn't exist
            key = dirpath+'_'+z_stack
            
            if '_FF_' not in key:
                if dapi_ch in channel and cycle_cond in condition:
                    if key not in img_ch:
                        img_ch[key] = {}
                    img_dapi = skimage.io.imread(os.path.join(dirpath,name))
                    img_dapi = (img_dapi/256).astype('uint8')
                    img_ch[key]['dapi'] = img_dapi 

                if pha_ch in channel and cycle_cond in condition:
                    if key not in img_ch:
                        img_ch[key] = {}
                    img_pha = skimage.io.imread(os.path.join(dirpath,name))
                    img_pha = (img_pha/256).astype('uint8')
                    img_ch[key]['pha'] = img_pha 
                if wga_ch in channel and cycle_cond in condition:
                    if key not in img_ch:
                        img_ch[key] = {}
                    img_wga = skimage.io.imread(os.path.join(dirpath,name))
                    img_wga = (img_wga/256).astype('uint8')
                    img_ch[key]['wga'] = img_wga
              


In [4]:
# Generate composite image from Dapi and phallodin channel
imgs = []

for key in img_ch.keys():                
    data = np.concatenate((img_ch[key]['pha'][:,:,None], img_ch[key]['wga'][:,:,None]), axis=2)
    data = np.concatenate((data, img_ch[key]['dapi'][:,:,None]), axis=2)
    img = convert_to_rgb(data)
    imgs.append(img)

In [5]:
# plt.figure(figsize=(15,15))
# plt.axis('off')
# plt.imshow(img)

# Segmentation with cellpose

In [4]:
import mxnet as mx 
from cellpose import utils

use_GPU = utils.use_gpu()
print('GPU activated? %d'%use_GPU)

from cellpose import models

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto', device = mx.gpu(0))

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
channels = [1,3]

** CUDA version installed and working. **
GPU activated? 1
cellpose_residual_on_style_on_concatenation_off


In [ ]:
masks, flows, styles, diams = model.eval(imgs, diameter=200, flow_threshold=None, channels=channels)

In [ ]:
# DISPLAY RESULTS
from cellpose import plot

nimg = len(imgs)
for idx in range(nimg):
    maski = masks[idx]
    flowi = flows[idx][0]

    fig = plt.figure(figsize=(25,15))
    plot.show_segmentation(fig, imgs[idx], maski, flowi, channels=channels)
    plt.tight_layout()
#     plt.savefig(str(idx)+'.png')
    plt.show()